Задание 1.
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.                  

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово.                      
Эти слова определяем в начале кода в переменной, например:KEYWORDS = ['python', 'парсинг']                             
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).                
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>                                    

Дополнительная часть (необязательная)                     
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.                    
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.                              
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
import json

In [2]:
import pandas as pd

In [86]:
KEYWORDS = ['python', 'парсинг']

In [78]:
URL = 'https://habr.com/ru/all/'

<Response [200]>

In [3]:
from bs4 import BeautifulSoup

In [94]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests

def links (URL, KEYWORDS, pages):
    a_list = []
    param = {'q': ' OR '.join(KEYWORDS)}
    for i in range(1, pages +1):
        res = requests.get(URL + str(i), param)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text,'html.parser')
        link_article = soup.find_all('h2', class_='post__title')
        a_list += list(map(lambda x: x.find('a', class_='post__title_link').get('href'), link_article))
    return a_list

URL = 'https://habr.com/ru/search/page'
KEYWORDS = ['python', 'парсинг']
pages = 1
all_links = links(URL, KEYWORDS, pages)
#print(all_links)
habr_search = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'),dayfirst=True)
    text = soup.find('div', id='post-content-body').text
    row = {'date': date, 'link': link, 'text': text}
    habr_search = pd.concat([habr_search, pd.DataFrame([row])])
habr_search.head()

,date,link,text
0,2020-07-31 07:00:00+00:00,https://habr.com/ru/post/512208/,В предыдущей статье в рамках коммерческого про...
0,2018-02-26 06:23:00+00:00,https://habr.com/ru/post/349860/,\nРегулярные выражения в Python от простого к ...
0,2016-03-03 13:39:00+00:00,https://habr.com/ru/company/pt/blog/210772/,В нашем проекте PT Application Inspector реали...
0,2018-06-13 17:26:00+00:00,https://habr.com/ru/post/414043/,"Hello Habr! Эта моя первая статья на Хабре, и ..."
0,2011-06-14 15:24:00+00:00,https://habr.com/ru/post/121815/,Статья описывает разработку скрипта на языке P...


Задание 2.                       
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck.                  
Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]                               
В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>                   
Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.                                 

In [4]:
import json
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests

EMAIL = ['xxz@mail.ru', 'yyz@mail.ru']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breach..'
request_mail = {"emailAddresses": EMAIL}
header_params = {'Vaar-Header-App-Build-Version': '1.0.0',
                'Vaar-Header-App-Product': 'hackcheck-web-avast',
                'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
                'Vaar-Version': '0'}
response = requests.post(URL, data=json.dumps(request_mail), headers=header_params).json()
df = pd.DataFrame()
for email in response['summary'].keys():
    for breach_id in response['summary'][email]['breaches']:
        breach = response['breaches'][str(breach_id)]['publishDate']
        row = pd.DataFrame([{
            'email': email,
            'date': response['breaches'][str(breach_id)]['publishDate'],
            'source': response['breaches'][str(breach_id)]['site'],
            'description': response['breaches'][str(breach_id)]['description']}])
        df = pd.concat([df, row])
display(df.head(100))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)